In [1]:
!pip install afinn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for afinn: filename=afinn-0.1-py3-none-any.whl size=53429 sha256=848c32c112bb85e80c684e7276cd859c794523d9fb24fc6e5b4ac4bde2b7e03a
  Stored in directory: /root/.cache/pip/wheels/b0/05/90/43f79196199a138fb486902fceca30a2d1b5228e6d2db8eb90
Successfully built afinn


In [7]:
from afinn import Afinn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier, BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

# Sample dataset (replace with your own dataset)
train = pd.read_csv('/kaggle/input/hybridapproach2/train/train.csv')
test = pd.read_csv('/kaggle/input/hybridapproach2/test/test.csv')

# Initialize AFINN sentiment analyzer
afinn = Afinn()

# Perform sentiment analysis using AFINN and add scores to dataset
train['afinn_score'] = train['review'].apply(lambda x: afinn.score(x))
test['afinn_score'] = test['review'].apply(lambda x: afinn.score(x))

# Vectorize the text data using CountVectorizer
count_vectorizer = CountVectorizer(max_features=5000)
train_count = count_vectorizer.fit_transform(train['review'])
test_count = count_vectorizer.transform(test['review'])

# Initialize classifiers
models = {
    "Logistic Regression": LogisticRegression(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Gradient Boosting": GradientBoostingClassifier(),
    "Neural Network": MLPClassifier(max_iter=1000),
    "KNN": KNeighborsClassifier(),
}

# Train and evaluate each model
for name, model in models.items():
    model.fit(train_count, train['sentiment'])
    predictions = model.predict(test_count)
    accuracy = accuracy_score(test['sentiment'], predictions)
    print(f"{name} Accuracy: {accuracy}")
    print(f"{name} Classification Report:")
    print(classification_report(test['sentiment'], predictions))


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression Accuracy: 0.87885
Logistic Regression Classification Report:
              precision    recall  f1-score   support

    negative       0.88      0.87      0.88      9935
    positive       0.87      0.89      0.88     10065

    accuracy                           0.88     20000
   macro avg       0.88      0.88      0.88     20000
weighted avg       0.88      0.88      0.88     20000

Decision Tree Accuracy: 0.7153
Decision Tree Classification Report:
              precision    recall  f1-score   support

    negative       0.71      0.71      0.71      9935
    positive       0.72      0.72      0.72     10065

    accuracy                           0.72     20000
   macro avg       0.72      0.72      0.72     20000
weighted avg       0.72      0.72      0.72     20000

Random Forest Accuracy: 0.8439
Random Forest Classification Report:
              precision    recall  f1-score   support

    negative       0.84      0.85      0.84      9935
    positive       0

In [ ]:
# Stacking
estimators = [(name, model) for name, model in models.items()]
stacking_model = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())
stacking_model.fit(train_count, train['sentiment'])
stacking_predictions = stacking_model.predict(test_count)
stacking_accuracy = accuracy_score(test['sentiment'], stacking_predictions)
print(f"Stacking Accuracy: {stacking_accuracy}")
print("Stacking Classification Report:")
print(classification_report(test['sentiment'], stacking_predictions))

# Voting Classifier
voting_model = VotingClassifier(estimators=estimators)
voting_model.fit(train_count, train['sentiment'])
voting_predictions = voting_model.predict(test_count)
voting_accuracy = accuracy_score(test['sentiment'], voting_predictions)
print(f"Voting Classifier Accuracy: {voting_accuracy}")
print("Voting Classifier Classification Report:")
print(classification_report(test['sentiment'], voting_predictions))

# Bagging Classifier
bagging_model = BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=10, random_state=42)
bagging_model.fit(train_count, train['sentiment'])
bagging_predictions = bagging_model.predict(test_count)
bagging_accuracy = accuracy_score(test['sentiment'], bagging_predictions)
print(f"Bagging Classifier Accuracy: {bagging_accuracy}")
print("Bagging Classifier Classification Report:")
print(classification_report(test['sentiment'], bagging_predictions))

In [8]:
from afinn import Afinn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier, BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('punkt')

# Sample dataset (replace with your own dataset)
train = pd.read_csv('/kaggle/input/hybridapproach2/train/train.csv')
test = pd.read_csv('/kaggle/input/hybridapproach2/test/test.csv')

# Remove stop words and tokenize the reviews
stop_words = set(stopwords.words('english'))
train['review'] = train['review'].apply(lambda x: ' '.join([word.lower() for word in word_tokenize(x) if word.lower() not in stop_words]))
test['review'] = test['review'].apply(lambda x: ' '.join([word.lower() for word in word_tokenize(x) if word.lower() not in stop_words]))

# Initialize AFINN sentiment analyzer
afinn = Afinn()

# Perform sentiment analysis using AFINN and add scores to dataset
train['afinn_score'] = train['review'].apply(lambda x: afinn.score(x))
test['afinn_score'] = test['review'].apply(lambda x: afinn.score(x))

# Vectorize the text data using CountVectorizer
count_vectorizer = CountVectorizer(max_features=5000)
train_count = count_vectorizer.fit_transform(train['review'])
test_count = count_vectorizer.transform(test['review'])

# Initialize classifiers
models = {
    "Logistic Regression": LogisticRegression(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Gradient Boosting": GradientBoostingClassifier(),
    "Neural Network": MLPClassifier(max_iter=1000),
    "KNN": KNeighborsClassifier(),
}

# Train and evaluate each model
for name, model in models.items():
    model.fit(train_count, train['sentiment'])
    predictions = model.predict(test_count)
    accuracy = accuracy_score(test['sentiment'], predictions)
    print(f"{name} Accuracy: {accuracy}")
    print(f"{name} Classification Report:")
    print(classification_report(test['sentiment'], predictions))


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression Accuracy: 0.8696
Logistic Regression Classification Report:
              precision    recall  f1-score   support

    negative       0.87      0.87      0.87      9935
    positive       0.87      0.87      0.87     10065

    accuracy                           0.87     20000
   macro avg       0.87      0.87      0.87     20000
weighted avg       0.87      0.87      0.87     20000

Decision Tree Accuracy: 0.7215
Decision Tree Classification Report:
              precision    recall  f1-score   support

    negative       0.72      0.73      0.72      9935
    positive       0.73      0.72      0.72     10065

    accuracy                           0.72     20000
   macro avg       0.72      0.72      0.72     20000
weighted avg       0.72      0.72      0.72     20000

Random Forest Accuracy: 0.84825
Random Forest Classification Report:
              precision    recall  f1-score   support

    negative       0.84      0.86      0.85      9935
    positive       0

In [ ]:
# Stacking
estimators = [(name, model) for name, model in models.items()]
stacking_model = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())
stacking_model.fit(train_count, train['sentiment'])
stacking_predictions = stacking_model.predict(test_count)
stacking_accuracy = accuracy_score(test['sentiment'], stacking_predictions)
print(f"Stacking Accuracy: {stacking_accuracy}")
print("Stacking Classification Report:")
print(classification_report(test['sentiment'], stacking_predictions))

# Voting Classifier
voting_model = VotingClassifier(estimators=estimators)
voting_model.fit(train_count, train['sentiment'])
voting_predictions = voting_model.predict(test_count)
voting_accuracy = accuracy_score(test['sentiment'], voting_predictions)
print(f"Voting Classifier Accuracy: {voting_accuracy}")
print("Voting Classifier Classification Report:")
print(classification_report(test['sentiment'], voting_predictions))

# Bagging Classifier
bagging_model = BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=10, random_state=42)
bagging_model.fit(train_count, train['sentiment'])
bagging_predictions = bagging_model.predict(test_count)
bagging_accuracy = accuracy_score(test['sentiment'], bagging_predictions)
print(f"Bagging Classifier Accuracy: {bagging_accuracy}")
print("Bagging Classifier Classification Report:")
print(classification_report(test['sentiment'], bagging_predictions))
